# Grupo 11 - TP 2
## XGBoost model
### Integrantes:
* Blas Chuc - 110253
* Franco Rodriguez - 108799
* Helen Chen - 110195
* Tomas Caporaletti - 108598

# Primera parte: Preprocesamiento y entrenamiento de XGBoost

## 1. Preprocesamiento de texto

En esta sección vamos a preparar las reseñas en español (`review_es`) para que puedan ser
utilizadas por los diferentes modelos de clasificación.

Trabajaremos con las siguientes etapas:

1. Carga de datos y exploración básica.
2. Detección de idioma y filtrado (quedarnos con español).
3. Limpieza básica de texto.
4. Tokenización y lematización en español (spaCy).
5. Vectorización (Bag of Words y TF–IDF) con filtros por frecuencia.


## 1.1 Carga de datos y exploración básica.
Importamos dependencias, cargamos los datos y obtenemos un poco de informacion de los mismos.

In [ ]:
# Descarga de train y test

!pip install gdown --quiet

# IDs reales de Google Drive
ID_TRAIN = "1eVXrJ4w7Gn6FZI7gl-GgucHTmw9105th"
ID_TEST  = "1KNzJ7RtqGMAQEEWZqrQ8G9j8yim1GNi1"

print("Descargando train.csv ...")
!gdown --id $ID_TRAIN -O train.csv

print("Descargando test.csv ...")
!gdown --id $ID_TEST -O test.csv

print("\n Archivos descargados correctamente.")

Descargando train.csv ...
/usr/local/lib/python3.12/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1eVXrJ4w7Gn6FZI7gl-GgucHTmw9105th
To: /content/train.csv
100% 72.2M/72.2M [00:01<00:00, 43.1MB/s]
Descargando test.csv ...
/usr/local/lib/python3.12/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1KNzJ7RtqGMAQEEWZqrQ8G9j8yim1GNi1
To: /content/test.csv
100% 11.1M/11.1M [00:00<00:00, 207MB/s]

 Archivos descargados correctamente.


In [ ]:
import pandas as pd
import numpy as np

# Para preprocesamiento de texto
import re
import nltk
from nltk.corpus import stopwords

# Para idioma
!pip install langid --quiet
import langid

# Para lematización en español
!python -m spacy download es_core_news_sm > /dev/null
import spacy

# Vectorizadores y split
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# Cargamos los datasets (los que descargó gdown)
train_df = pd.read_csv("train.csv")
test_df  = pd.read_csv("test.csv")

print("Train - filas:", train_df.shape[0], "columnas:", train_df.shape[1])
print("Test  - filas:", test_df.shape[0], "columnas:", test_df.shape[1])

print("\nColumnas del train:")
print(train_df.columns)

print("\nDistribución de clases en train:")
print(train_df["sentimiento"].value_counts())

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 21.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
Train - filas: 50000 columnas: 3
Test  - filas: 8599 columnas: 2

Columnas del train:
Index(['ID', 'review_es', 'sentimiento'], dtype='object')

Distribución de clases en train:
sentimiento
positivo    25000
negativo    25000
Name: count, dtype: int64


### 1.2 Detección de idioma

Aunque las reseñas deberían estar en español, verificamos los idiomas presentes en los
conjuntos de `train` y `test`, para eventualmente filtrar sólo las reseñas en español en el
conjunto de entrenamiento.


In [ ]:
def identify_language(text):
    return langid.classify(str(text))[0]

train_df["language"] = train_df["review_es"].apply(identify_language)
test_df["language"]  = test_df["review_es"].apply(identify_language)

print("Idiomas en TRAIN:")
print(train_df["language"].value_counts())
print("\nIdiomas en TEST:")
print(test_df["language"].value_counts())

# Nos quedamos solo con reseñas en español en train
train_df = train_df[train_df["language"] == "es"].copy()

# Ya no necesitamos la columna language
train_df  = train_df.drop(columns=["language"])
test_df   = test_df.drop(columns=["language"])

print("\nTamaño de train luego de filtrar a español:", train_df.shape)


Idiomas en TRAIN:
language
es    48179
en     1817
gl        2
pt        1
id        1
Name: count, dtype: int64

Idiomas en TEST:
language
es    8597
ga       1
la       1
Name: count, dtype: int64

Tamaño de train luego de filtrar a español: (48179, 3)


### 1.3 Limpieza básica de texto

Aplicamos una limpieza estándar:
- Pasar todo a minúsculas.
- Eliminar etiquetas HTML.
- Quitar caracteres no alfabéticos (números, símbolos, etc.).
- Normalizar espacios.

### 1.4 Tokenización y lematización en español

Usaremos **spaCy** con el modelo `es_core_news_sm` para:
- dividir el texto en tokens,
- lematizar cada palabra (obtener su forma base),
- eliminar stopwords en español.

El resultado será texto "normalizado" donde:
- *"películas"*, *"película"* → `película`
- *"buenísima"*, *"buena"* → `bueno`
- *"corriendo"* → `correr`

In [ ]:
nltk.download("stopwords")
stop_es = set(stopwords.words("spanish"))

nlp = spacy.load("es_core_news_sm")

def clean_text(text):
    text = str(text).lower()
    text = re.sub(r"<[^>]+>", " ", text)            # eliminar HTML
    text = re.sub(r"[^a-záéíóúüñ\s]", " ", text)    # solo letras y espacios
    text = re.sub(r"\s+", " ", text)                # espacios múltiples
    return text.strip()

def tokenize_and_lemmatize(text):
    text = clean_text(text)
    doc = nlp(text)

    tokens = [
        token.lemma_.lower()
        for token in doc
        if token.is_alpha               # solo tokens alfabéticos
        and token.lemma_.lower() not in stop_es
        and len(token.lemma_) > 2       # evitar palabras muy cortas
    ]
    return tokens

def preprocess_to_string(text):
    """
    Aplica limpieza + spaCy y devuelve un string con lemas separados por espacios.
    Útil si queremos guardar la versión procesada.
    """
    return " ".join(tokenize_and_lemmatize(text))



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


### 1.5 Generación de texto procesado y división train/valid

Creamos una versión procesada de las reseñas (`review_proc`) y dividimos el conjunto de
entrenamiento en *train* y *validación* respetando el balance de clases.

In [ ]:
# Texto crudo y etiquetas
X_raw = train_df["review_es"]
y     = train_df["sentimiento"]

# Mapeo a binario
y_bin = y.map({"negativo": 0, "positivo": 1})

print("Iniciando preprocesamiento")
print("Preprocesando datos crudos de train")
X_proc = X_raw.apply(preprocess_to_string)

# Guardamos también el test preprocesado
print("Preprocesando datos crudos de test")
X_test_raw  = test_df["review_es"]
X_test_proc = X_test_raw.apply(preprocess_to_string)

# División en train/valid
# Dividimos train en una parte para entrenamiento y otra parte para testeo
print("Separando datos de train en entrenamiento y testeo")
X_train_text, X_val_text, y_train, y_val = train_test_split(
    X_proc,
    y,
    test_size=0.2,   # 20% del dataset train se va a usar para testeo
    random_state=42,
    stratify=y
)

print("Tamaño X_train:", len(X_train_text))
print("Tamaño X_val  :", len(X_val_text))


Iniciando preprocesamiento
Preprocesando datos crudos de train
Preprocesando datos crudos de test
Separando datos de train en entrenamiento y testeo
Tamaño X_train: 38543
Tamaño X_val  : 9636


### 1.6 Vectorización: Bag-of-Words y TF–IDF

A partir del texto ya limpiado y lematizado, generamos representaciones numéricas:

- **Bag-of-Words (CountVectorizer)**: matriz de conteos de palabras.
- **TF–IDF (TfidfVectorizer)**: repondera los conteos según qué tan específica es cada palabra.

En ambos casos:
- `min_df = 10` → se descartan palabras que aparecen en menos de 10 reseñas.
- `max_df = 0.9` → se descartan palabras que aparecen en más del 90% de las reseñas.

| Modelo | Qué representa el vector          |
| ------ | --------------------------------- |
| BOW    | Conteo de palabras                |
| TF-IDF | Importancia ponderada de palabras |



In [ ]:

# 1.6.1 Bag-of-Words
bow_vectorizer = CountVectorizer(
    preprocessor=None,          # ya preprocesamos antes
    tokenizer=str.split,        # nuestros textos son "lemma1 lemma2 ..."
    min_df=10,  # Aparecen en menos de diez reviews
    max_df=0.9  # Aparece en no mas del 90%
)

X_train_bow = bow_vectorizer.fit_transform(X_train_text)
X_val_bow   = bow_vectorizer.transform(X_val_text)

print("BOW - Textos en entrenamiento:", X_train_bow.shape[0])
print("BOW - Cantidad de palabras distintas:", X_train_bow.shape[1])

# Top 20 palabras más frecuentes (BOW)
contador_palabras = np.array(X_train_bow.sum(axis=0)).flatten()
palabras = bow_vectorizer.get_feature_names_out()

freq_df = pd.DataFrame({"palabra": palabras, "frecuencia": contador_palabras})
freq_df = freq_df.sort_values(by="frecuencia", ascending=False)
print("\nTop 20 palabras (BOW):")
display(freq_df.head(20))


# 1.6.2 TF-IDF
tfidf_vectorizer = TfidfVectorizer(
    preprocessor=None,
    tokenizer=str.split,
    min_df=10,
    max_df=0.9,
    ngram_range=(1, 2)   # unigrams + bigrams
)

X_train_tfidf = tfidf_vectorizer.fit_transform(X_train_text)
X_val_tfidf   = tfidf_vectorizer.transform(X_val_text)

print("\nTF-IDF - Textos en entrenamiento:", X_train_tfidf.shape[0])
print("TF-IDF - Cantidad de features:", X_train_tfidf.shape[1])


/usr/local/lib/python3.12/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


BOW - Textos en entrenamiento: 38543
BOW - Cantidad de palabras distintas: 18418

Top 20 palabras (BOW):


,palabra,frecuencia
12993,película,143807
8140,haber,75600
16830,tener,53047
17709,ver,52199
8159,hacer,49322
13413,poder,48278
18386,él,37921
16102,solo,24255
593,alguno,23590
8509,historia,22415


/usr/local/lib/python3.12/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(



TF-IDF - Textos en entrenamiento: 38543
TF-IDF - Cantidad de features: 64066


## 2. XGBoost Model TF-IDF

En esta sección vamos a entrenar un modelo de clasificación utilizando **XGBoost** sobre las
reseñas ya preprocesadas y vectorizadas con **TF–IDF**.

Trabajaremos en cuatro etapas:

1. Preparación de los datos para XGBoost (etiquetas numéricas y matrices TF–IDF).
2. Entrenamiento de un modelo base para verificar el pipeline.
3. Búsqueda de hiperparámetros (RandomizedSearchCV) optimizando F1.
4. Entrenamiento final con todos los datos de `train.csv` y generación del archivo de
   submission para Kaggle.


2.1 Preparación de datos para XGBoost

In [ ]:
!pip install xgboost --quiet

from xgboost import XGBClassifier
from sklearn.metrics import f1_score, classification_report

# y_train y y_val están como "positivo"/"negativo"
# Para XGBoost trabajamos con etiquetas binarias: 0 = negativo, 1 = positivo
y_train_bin = y_train.map({"negativo": 0, "positivo": 1})
y_val_bin   = y_val.map({"negativo": 0, "positivo": 1})

print("Tamaño X_train_tfidf:", X_train_tfidf.shape)
print("Tamaño X_val_tfidf  :", X_val_tfidf.shape)
print("Etiquetas (train)   :")
print(y_train_bin.value_counts())

Tamaño X_train_tfidf: (38543, 64066)
Tamaño X_val_tfidf  : (9636, 64066)
Etiquetas (train)   :
sentimiento
0    19305
1    19238
Name: count, dtype: int64


### 2.2 Modelo base de XGBoost

Antes de hacer una búsqueda de hiperparámetros, entrenamos un modelo base de XGBoost con
parámetros razonables. El objetivo es:

- Verificar que el pipeline `texto → TF–IDF → XGBoost` funciona sin errores.
- Obtener una primera referencia de desempeño (F1 en el conjunto de validación).

### ¿Por qué conviene hacemos esto antes del modelo final?

1. Porque nos indica:
* ¿Está bien el preprocesamiento?
* ¿La vectorización TF-IDF funciona?
* ¿Las dimensiones de las matrices son correctas?
* ¿Las etiquetas están bien mapeadas (0/1)?

2. Tener un “piso” de rendimiento
Supongamos que el modelo base da:
* F1 = 0.87 en validación.

  Cuando tengamos el modelo final, vamos a querer mejorar eso. Si el modelo con hiperparámetros “óptimos” da F1 = 0.86, algo raro está pasando.

3. Chequeo de tiempo de entrenamiento
* Se entrena rápido o tarda mucho.
* Intentar un RandomizedSearchCV con N combinaciones o si conviene achicar el espacio de búsqueda.

In [ ]:
xgb_base = XGBClassifier(
    objective="binary:logistic",
    eval_metric="logloss",  # métrica interna para el entrenamiento
    random_state=42,
    tree_method="hist",     # más rápido en CPU
)

xgb_base.fit(X_train_tfidf, y_train_bin)

# Predicciones en validación
y_val_pred_bin_base = xgb_base.predict(X_val_tfidf)

print("F1 (validación, modelo base):", f1_score(y_val_bin, y_val_pred_bin_base))
print("\nReporte de clasificación (modelo base):\n")
print(classification_report(y_val_bin, y_val_pred_bin_base, target_names=["negativo", "positivo"]))

F1 (validación, modelo base): 0.8444627080336963

Reporte de clasificación (modelo base):

              precision    recall  f1-score   support

    negativo       0.85      0.83      0.84      4826
    positivo       0.83      0.85      0.84      4810

    accuracy                           0.84      9636
   macro avg       0.84      0.84      0.84      9636
weighted avg       0.84      0.84      0.84      9636



### 2.3 Búsqueda de hiperparámetros (RandomizedSearchCV)

Para mejorar el desempeño del modelo, realizamos una búsqueda aleatoria de hiperparámetros
sobre un espacio razonable, utilizando **RandomizedSearchCV** con validación cruzada.

- Métrica de optimización: **F1** (clase positiva).
- Validación cruzada: 4 folds.
- Número de combinaciones probadas: 5.

Basicamente, entrenamos multiples modelos con distintos hiperparametros para obtener los mejores, RandomizedSearchCV:
* Toma 5 combinaciones aleatorias (porque n_iter=5)
* Por cada combinación entrena un modelo XGBoost
* Y como folds (cv) = 4, hace esto:

  Por cada combinación de hiperparametros:

   * divide el dataset en 4 partes

   * entrena 4 modelos (cada vez entrenando en 3 partes y validando en 1 distinta)
   calcula 4 F1

   * promedia los F1 para decidir qué tan buena es esa combinación, selecciona la combinación que obtiene el mejor desempeño promedio.


**Tardo 42 minutos en correr**

¿Por que tarda tanto?

5 combinaciones × 4 modelos cada una = 20 modelos XGBoost entrenados

Promedio de tiempo en entrenar un modelo = 3 minutos

20 modelos x 3 minutos/modelo = 60 min

(Para que tarde menos podemos bajar la cantidad de combinaciones o reducir el espacio de hiperparametros/folds/cv)

INICIALMENTE probamos con 20 iteraciones, pero vimos que tardaria 4 horas aproximadamente y disminuimos.


In [ ]:
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

param_dist = {
    "n_estimators": [250, 350],
    "max_depth": [3, 4],
    "learning_rate": [0.05, 0.1],
    "subsample": [0.8, 1.0],
    "colsample_bytree": [0.8, 1.0],
    "gamma": [0, 1, 2],
    "reg_lambda": [1.0, 1.5],
    "reg_alpha": [0.0, 0.1],
}

xgb_clf = XGBClassifier(
    objective="binary:logistic",
    eval_metric="logloss",
    random_state=42,
    tree_method="hist",
)

xgb_search = RandomizedSearchCV(
    estimator=xgb_clf,
    param_distributions=param_dist,
    n_iter=5,            # número de combinaciones a probar
    scoring="f1",         # F1 para la clase positiva
    cv=4,
    verbose=1,
    n_jobs=-1
)

xgb_search.fit(X_train_tfidf, y_train_bin)

print("\nMejores hiperparámetros encontrados:")
print(xgb_search.best_params_)
print("Mejor F1 (CV):", xgb_search.best_score_)


Fitting 4 folds for each of 5 candidates, totalling 20 fits

Mejores hiperparámetros encontrados:
{'subsample': 0.8, 'reg_lambda': 1.0, 'reg_alpha': 0.0, 'n_estimators': 250, 'max_depth': 4, 'learning_rate': 0.1, 'gamma': 1, 'colsample_bytree': 1.0}
Mejor F1 (CV): 0.8472315263412213


El mejor F1 (CV) obtenido fue de: 0.8472315263412213

Mientras que el F1 del modelo base de validacion fue de: 0.8444627080336963

Es decir, un 0.003 de diferencia, por lo que podemos inferir que:
* parametros de XGBClassifier ya eran mas que razonables
* tuvimos un solido preprocesamiento
* el dataset esta bastante "limpio"

#### 2.3.1 Evaluación del mejor modelo en el conjunto de validación

Una vez finalizada la búsqueda de hiperparámetros, evaluamos el mejor modelo (`best_estimator_`)
sobre nuestro conjunto de validación reservado anteriormente (20% del `train.csv`).


In [ ]:
best_xgb = xgb_search.best_estimator_

y_val_pred_bin = best_xgb.predict(X_val_tfidf)

print("F1 (validación, XGBoost tuned):", f1_score(y_val_bin, y_val_pred_bin))
print("\nReporte de clasificación (XGBoost tuned):\n")
print(classification_report(y_val_bin, y_val_pred_bin, target_names=["negativo", "positivo"]))


F1 (validación, XGBoost tuned): 0.8430475027972739

Reporte de clasificación (XGBoost tuned):

              precision    recall  f1-score   support

    negativo       0.86      0.82      0.84      4826
    positivo       0.83      0.86      0.84      4810

    accuracy                           0.84      9636
   macro avg       0.84      0.84      0.84      9636
weighted avg       0.84      0.84      0.84      9636



### 2.4 Entrenamiento final

Una vez seleccionados los hiperparámetros óptimos, reentrenamos el modelo utilizando **todo
el conjunto de entrenamiento (`train.csv`)** para aprovechar al máximo la información disponible.

Pasos:

1. Volver a vectorizar **todas** las reseñas procesadas de train (`X_proc`) con TF–IDF.
2. Vectorizar también las reseñas preprocesadas de test (`X_test_proc`) con el mismo vectorizador.
3. Entrenar un nuevo modelo XGBoost con los mejores hiperparámetros sobre todo el train.
4. Predecir los sentimientos del conjunto `test.csv`.
5. Construir el archivo `submission_xgboost_tfidf.csv` con las columnas `ID` y `sentimiento`.


In [ ]:
# Etiquetas binarias sobre TODO el train
y_bin_full = y.map({"negativo": 0, "positivo": 1})

# Vectorizador TF-IDF entrenado con TODO el train procesado
tfidf_vectorizer_full = TfidfVectorizer(
    preprocessor=None,
    tokenizer=str.split,
    min_df=10,
    max_df=0.9,
    ngram_range=(1, 2)
)

print("Vectorizando TODO el train (TF-IDF)...")
X_full_tfidf   = tfidf_vectorizer_full.fit_transform(X_proc)
print("Vectorizando TODO el test (TF-IDF)...")
X_kaggle_tfidf = tfidf_vectorizer_full.transform(X_test_proc)

# Recuperamos los mejores hiperparámetros encontrados antes
best_params = xgb_search.best_params_

xgb_final = XGBClassifier(
    objective="binary:logistic",
    eval_metric="logloss",
    random_state=42,
    tree_method="hist",
    **best_params
)

print("Entrenando XGBoost final sobre TODO el train...")
xgb_final.fit(X_full_tfidf, y_bin_full)

# Predicciones sobre el test de Kaggle
y_test_pred_bin = xgb_final.predict(X_kaggle_tfidf)

# Convertimos de 0/1 a "negativo"/"positivo"
pred_labels = np.where(y_test_pred_bin == 1, "positivo", "negativo")

submission = pd.DataFrame({
    "ID": test_df["ID"],
    "sentimiento": pred_labels
})

submission_filename = "submission_xgboost_tfidf.csv"
submission.to_csv(submission_filename, index=False)

print(f"Archivo de submission generado: {submission_filename}")
submission.head()

Vectorizando TODO el train (TF-IDF)...


/usr/local/lib/python3.12/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Vectorizando TODO el test (TF-IDF)...
Entrenando XGBoost final sobre TODO el train...
Archivo de submission generado: submission_xgboost_tfidf.csv


,ID,sentimiento
0,60000,negativo
1,60001,positivo
2,60002,negativo
3,60003,negativo
4,60004,negativo


### 2.5 (Opcional) Guardado del modelo y del vectorizador

Guardamos el modelo XGBoost final y el vectorizador TF–IDF utilizando `joblib`, de modo que se puedan cargar posteriormente sin necesidad de reentrenar.


In [ ]:
from joblib import dump

dump(xgb_final, "modelo_xgboost_tfidf.joblib")
dump(tfidf_vectorizer_full, "vectorizador_tfidf_xgb.joblib")

print("Modelo y vectorizador guardados correctamente.")


Modelo y vectorizador guardados correctamente.


## 3. XGBoost con Bag-of-Words (BOW)

En esta sección repetimos el experimento de XGBoost pero utilizando la representación
Bag-of-Words (conteo de palabras) en lugar de TF–IDF.

Volvemos a usar las mismas reseñas preprocesadas (`X_train_text`, `X_val_text`) y las mismas
etiquetas (`y_train`, `y_val`), pero ahora el modelo recibe como entrada las matrices:

- `X_train_bow` (entrenamiento)
- `X_val_bow` (validación)

y finalmente vectorizamos todo el `train.csv` y `test.csv` con un `CountVectorizer` entrenado
sobre todas las reseñas preprocesadas.


In [ ]:
# ============================================
# 3.1 Preparación de datos para XGBoost (BOW)
# ============================================

from xgboost import XGBClassifier
from sklearn.metrics import f1_score, classification_report

# Reutilizamos y_train, y_val de antes
y_train_bin_bow = y_train.map({"negativo": 0, "positivo": 1})
y_val_bin_bow   = y_val.map({"negativo": 0, "positivo": 1})

print("Tamaño X_train_bow:", X_train_bow.shape)
print("Tamaño X_val_bow  :", X_val_bow.shape)
print("Etiquetas (train) :")
print(y_train_bin_bow.value_counts())


Tamaño X_train_bow: (38543, 18418)
Tamaño X_val_bow  : (9636, 18418)
Etiquetas (train) :
sentimiento
0    19305
1    19238
Name: count, dtype: int64


3.1 Entrenamiento de un modelo base XGBoost BOW

In [ ]:
!pip install xgboost --quiet

xgb_base_bow = XGBClassifier(
    objective="binary:logistic",
    eval_metric="logloss",
    random_state=42,
    tree_method="hist",
)

xgb_base_bow.fit(X_train_bow, y_train_bin_bow)

y_val_pred_bin_base_bow = xgb_base_bow.predict(X_val_bow)

print("F1 (validación, modelo base BOW):", f1_score(y_val_bin_bow, y_val_pred_bin_base_bow))
print("\nReporte de clasificación (modelo base BOW):\n")
print(classification_report(y_val_bin_bow, y_val_pred_bin_base_bow, target_names=["negativo", "positivo"]))


F1 (validación, modelo base BOW): 0.8401770093650304

Reporte de clasificación (modelo base BOW):

              precision    recall  f1-score   support

    negativo       0.85      0.83      0.84      4826
    positivo       0.83      0.85      0.84      4810

    accuracy                           0.84      9636
   macro avg       0.84      0.84      0.84      9636
weighted avg       0.84      0.84      0.84      9636



3.2 RandomizedSearchCV pero usando BOW

Vamos a usar el mismo param_dist que definimos para TF-IDF y el mismo esquema de 5 combinaciones y cv=4.

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

param_dist_bow = {
    "n_estimators": [250, 350],
    "max_depth": [3, 4],
    "learning_rate": [0.05, 0.1],
    "subsample": [0.8, 1.0],
    "colsample_bytree": [0.8, 1.0],
    "gamma": [0, 1, 2],
    "reg_lambda": [1.0, 1.5],
    "reg_alpha": [0.0, 0.1],
}

xgb_clf_bow = XGBClassifier(
    objective="binary:logistic",
    eval_metric="logloss",
    random_state=42,
    tree_method="hist",
)

xgb_search_bow = RandomizedSearchCV(
    estimator=xgb_clf_bow,
    param_distributions=param_dist_bow,
    n_iter=5,          # igual que antes: 5 combinaciones
    scoring="f1",
    cv=4,
    verbose=1,
    n_jobs=-1
)

xgb_search_bow.fit(X_train_bow, y_train_bin_bow)

print("\nMejores hiperparámetros (BOW):")
print(xgb_search_bow.best_params_)
print("Mejor F1 (CV, BOW):", xgb_search_bow.best_score_)


Fitting 4 folds for each of 5 candidates, totalling 20 fits

Mejores hiperparámetros (BOW):
{'subsample': 0.8, 'reg_lambda': 1.0, 'reg_alpha': 0.0, 'n_estimators': 350, 'max_depth': 4, 'learning_rate': 0.1, 'gamma': 2, 'colsample_bytree': 0.8}
Mejor F1 (CV, BOW): 0.8501484335205565


3.3.1 Evaluar el mejor modelo BOW en validación

In [ ]:
best_xgb_bow = xgb_search_bow.best_estimator_

y_val_pred_bin_bow = best_xgb_bow.predict(X_val_bow)

print("F1 (validación, XGBoost BOW tuned):", f1_score(y_val_bin_bow, y_val_pred_bin_bow))
print("\nReporte de clasificación (XGBoost BOW tuned):\n")
print(classification_report(y_val_bin_bow, y_val_pred_bin_bow, target_names=["negativo", "positivo"]))


F1 (validación, XGBoost BOW tuned): 0.8470107307102708

Reporte de clasificación (XGBoost BOW tuned):

              precision    recall  f1-score   support

    negativo       0.86      0.83      0.84      4826
    positivo       0.83      0.86      0.85      4810

    accuracy                           0.84      9636
   macro avg       0.85      0.84      0.84      9636
weighted avg       0.85      0.84      0.84      9636



3.4 Entrenamiento final + submission BOW

In [ ]:
# Etiquetas binarias sobre TODO el train (reutilizamos y)
y_bin_full_bow = y.map({"negativo": 0, "positivo": 1})

# Vectorizador BOW entrenado con TODO el train procesado
bow_vectorizer_full = CountVectorizer(
    preprocessor=None,
    tokenizer=str.split,
    min_df=10,
    max_df=0.9
)

print("Vectorizando TODO el train (BOW)...")
X_full_bow   = bow_vectorizer_full.fit_transform(X_proc)
print("Vectorizando TODO el test (BOW)...")
X_kaggle_bow = bow_vectorizer_full.transform(X_test_proc)

# Mejores hiperparámetros encontrados para BOW
best_params_bow = xgb_search_bow.best_params_

xgb_final_bow = XGBClassifier(
    objective="binary:logistic",
    eval_metric="logloss",
    random_state=42,
    tree_method="hist",
    **best_params_bow
)

print("Entrenando XGBoost BOW final sobre TODO el train...")
xgb_final_bow.fit(X_full_bow, y_bin_full_bow)

# Predicciones sobre el test de Kaggle (BOW)
y_test_pred_bin_bow = xgb_final_bow.predict(X_kaggle_bow)

pred_labels_bow = np.where(y_test_pred_bin_bow == 1, "positivo", "negativo")

submission_bow = pd.DataFrame({
    "ID": test_df["ID"],
    "sentimiento": pred_labels_bow
})

submission_filename_bow = "submission_xgboost_bow.csv"
submission_bow.to_csv(submission_filename_bow, index=False)

print(f"Archivo de submission generado (BOW): {submission_filename_bow}")
submission_bow.head()


Vectorizando TODO el train (BOW)...


/usr/local/lib/python3.12/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Vectorizando TODO el test (BOW)...
Entrenando XGBoost BOW final sobre TODO el train...
Archivo de submission generado (BOW): submission_xgboost_bow.csv


,ID,sentimiento
0,60000,negativo
1,60001,positivo
2,60002,negativo
3,60003,negativo
4,60004,negativo


3.5 Guardar modelo + vectorizador BOW

In [ ]:
from joblib import dump

dump(xgb_final_bow, "modelo_xgboost_bow.joblib")
dump(bow_vectorizer_full, "vectorizador_bow_xgb.joblib")

print("Modelo BOW y vectorizador BOW guardados correctamente.")


Modelo BOW y vectorizador BOW guardados correctamente.


# Segunda parte: Ensamble TF-IDF

## Ensamble de modelos (Naive Bayes + Random Forest + XGBoost)

En esta sección construimos un modelo de **ensamble** combinando tres modelos que ya fueron
entrenados previamente por distintos integrantes del grupo:

- Naive Bayes optimizado (`naive_bayes_optimizado.pkl`)
- Random Forest (`Random_Forest.pkl`)
- XGBoost con TF–IDF (`modelo_xgboost_tfidf.joblib` + `vectorizador_tfidf_xgb.joblib`)

Siguiendo la indicación del profesor, **no reentrenamos** estos modelos, sino que los usamos tal
como fueron guardados y los ensamblamos “por encima”.

Como cada modelo fue entrenado con su propio pipeline de preprocesamiento / vectorización,
no utilizamos `VotingClassifier` de sklearn (que requiere que todos compartan exactamente el
mismo `X`). En su lugar, hacemos un **ensamble manual por promediado de probabilidades**:

1. Para cada review de validación, pedimos a Naive Bayes, Random Forest y XGBoost la
   probabilidad de que la review sea *positiva*.
2. Promediamos las 3 probabilidades.
3. Si el promedio ≥ 0.5, clasificamos como *positivo*; si no, como *negativo*.

Evaluamos este ensamble sobre el mismo conjunto de validación que usamos para XGBoost y
comparamos el F1-score frente a los modelos individuales.


## 1.1 Cargar modelos preentrenados

In [ ]:
!pip install gdown --quiet

import gdown
import joblib
from joblib import load

# IDs de Google Drive (tomados de las URLs que pasaste)
ID_NB_OPT  = "15Ujr776-4Jvvxu78fpv1aSUfKKBv6nbC"   # naive_bayes_optimizado.pkl
ID_RF      = "1paViOtodbIqbn6AedMV6s8xEr8GFGnsR"   # Random_Forest.pkl
ID_XGB     = "1GqLlErfY34LdZs705oHJ2CnKSlzBa5bm"   # modelo_xgboost_tfidf.joblib
ID_XGB_VEC = "1g3rLHJpAvpZfy48i88dQdqrBAeZshrCt"   # vectorizador_tfidf_xgb.joblib

# Nombres de archivo locales (cómo se van a guardar en Colab)
NB_FILENAME      = "naive_bayes_optimizado.pkl"
RF_FILENAME      = "Random_Forest.pkl"
XGB_FILENAME     = "modelo_xgboost_tfidf.joblib"
XGB_VEC_FILENAME = "vectorizador_tfidf_xgb.joblib"

print("Descargando modelos preentrenados...")

gdown.download(id=ID_NB_OPT,  output=NB_FILENAME,      quiet=False)
gdown.download(id=ID_RF,      output=RF_FILENAME,      quiet=False)
gdown.download(id=ID_XGB,     output=XGB_FILENAME,     quiet=False)
gdown.download(id=ID_XGB_VEC, output=XGB_VEC_FILENAME, quiet=False)

print("\nDescarga completa.")


Descargando modelos preentrenados...


Downloading...
From: https://drive.google.com/uc?id=15Ujr776-4Jvvxu78fpv1aSUfKKBv6nbC
To: /content/naive_bayes_optimizado.pkl
100%|██████████| 49.9M/49.9M [00:00<00:00, 59.2MB/s]
Downloading...
From: https://drive.google.com/uc?id=1paViOtodbIqbn6AedMV6s8xEr8GFGnsR
To: /content/Random_Forest.pkl
100%|██████████| 12.9M/12.9M [00:00<00:00, 49.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=1GqLlErfY34LdZs705oHJ2CnKSlzBa5bm
To: /content/modelo_xgboost_tfidf.joblib
100%|██████████| 400k/400k [00:00<00:00, 85.3MB/s]
Downloading...
From: https://drive.google.com/uc?id=1g3rLHJpAvpZfy48i88dQdqrBAeZshrCt
To: /content/vectorizador_tfidf_xgb.joblib
100%|██████████| 2.08M/2.08M [00:00<00:00, 93.9MB/s]


Descarga completa.


In [ ]:
print("Cargando modelos...")

nb_model  = joblib.load(NB_FILENAME)
rf_model  = joblib.load(RF_FILENAME)
xgb_model = load(XGB_FILENAME)
tfidf_xgb = load(XGB_VEC_FILENAME)

print("Modelos cargados correctamente.")
print(type(nb_model))
print(type(rf_model))
print(type(xgb_model))
print(type(tfidf_xgb))


Cargando modelos...


/usr/lib/python3.12/pickle.py:1760: UserWarning: [23:15:13] WARNING: /workspace/src/collective/../data/../common/error_msg.h:83: If you are loading a serialized model (like pickle in Python, RDS in R) or
configuration generated by an older version of XGBoost, please export the model by calling
`Booster.save_model` from that version first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/stable/tutorials/saving_model.html

for more details about differences between saving model and serializing.

  setstate(state)


Modelos cargados correctamente.
<class 'sklearn.pipeline.Pipeline'>
<class 'sklearn.pipeline.Pipeline'>
<class 'xgboost.sklearn.XGBClassifier'>
<class 'sklearn.feature_extraction.text.TfidfVectorizer'>


## 1.2 Carga de datos

In [ ]:
import pandas as pd
!pip install gdown --quiet
import gdown

# IDs de Google Drive
ID_TRAIN = "1eVXrJ4w7Gn6FZI7gl-GgucHTmw9105th"
ID_TEST  = "1KNzJ7RtqGMAQEEWZqrQ8G9j8yim1GNi1"

print("Descargando train.csv ...")
gdown.download(id=ID_TRAIN, output="train.csv", quiet=False)

print("Descargando test.csv ...")
gdown.download(id=ID_TEST, output="test.csv", quiet=False)

print("\nLeyendo CSVs...")
train_df = pd.read_csv("train.csv")
test_df  = pd.read_csv("test.csv")

print("Train shape:", train_df.shape)
print("Test shape :", test_df.shape)
print(train_df.head(3))


Descargando train.csv ...


Downloading...
From: https://drive.google.com/uc?id=1eVXrJ4w7Gn6FZI7gl-GgucHTmw9105th
To: /content/train.csv
100%|██████████| 72.2M/72.2M [00:00<00:00, 199MB/s]


Descargando test.csv ...


Downloading...
From: https://drive.google.com/uc?id=1KNzJ7RtqGMAQEEWZqrQ8G9j8yim1GNi1
To: /content/test.csv
100%|██████████| 11.1M/11.1M [00:00<00:00, 57.1MB/s]



Leyendo CSVs...
Train shape: (50000, 3)
Test shape : (8599, 2)
   ID                                          review_es sentimiento
0   0  Uno de los otros críticos ha mencionado que de...    positivo
1   1  Una pequeña pequeña producción.La técnica de f...    positivo
2   2  Pensé que esta era una manera maravillosa de p...    positivo


## 2.1 Preprocesamiento estilo Naive Bayes / Random Forest

NB y RF fueron entrenados usando un pipeline de NLTK con stemming.
Para poder usar estos modelos en el ensamble, debemos aplicar el MISMO preprocesamiento para generar sus entradas.

En esta sección replicamos exactamente ese pipeline:

* convertir a minúsculas
* remover símbolos
* quitar stopwords
* aplicar stemming con SnowballStemmer

Esto produce X_nb_all y X_nb_test, que luego se usarán como entrada a NB y RF.

In [ ]:
import nltk, re
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

nltk.download('stopwords', quiet=True)

stemmer = SnowballStemmer('spanish')

# Stopwords en español, pero conservando algunas palabras de negación/matiz
stop_words_es = set(stopwords.words('spanish'))
palabras_a_mantener = {'no', 'ni', 'sin', 'pero', 'nada', 'poco', 'muy', 'nunca', 'jamás'}
stop_words_es = stop_words_es - palabras_a_mantener

def preprocess_text_stem(text):
    # Basado en los notebooks N1/N2 del grupo
    text = str(text).lower()
    text = re.sub(r'<[^>]+>', ' ', text)           # eliminar HTML
    text = re.sub(r'[^a-záéíóúñ\s]', '', text)    # solo letras y espacios
    words = text.split()

    cleaned_words = [
        stemmer.stem(word)
        for word in words
        if word not in stop_words_es and len(word) > 1
    ]

    return ' '.join(cleaned_words)

# Texto preprocesado "estilo NB/RF" para TODO el train y test
X_nb_all   = train_df["review_es"].apply(preprocess_text_stem)
X_nb_test  = test_df["review_es"].apply(preprocess_text_stem)

print("Ejemplo preprocesado (NB/RF):")
print(X_nb_all.iloc[0])


Ejemplo preprocesado (NB/RF):
critic mencion despues ver sol oz episodi enganch razon exact suced conmig primer cos golpe oz brutal escen violenci inconfi encuentr derech palabr conf no espectacul debil corazon tim espectacul no extra punzon respect drog sex violenci hardcor uso clasic palabr llam oz apod dad penitenciari segur maxim oswald centr principal ciud emerald seccion experimental prision tod celul frent vidri enfrent haci adentr privac no alta agend em city hog fariari musulman gangst latin cristian italian irlandes asi espos mir muert relacion peligr acuerd sombr nunc lej dir principal atract espectacul deb hech va espectacul no atrev olvidat imagen bonit pint audienci convencional olvid encant olvid romanc oz no met prim episodi vist sorprend tan desagrad surreal no pod dec list ello per observ desarroll gust oz acostumbr altos nivel violenci grafic no sol violenci sin injustici guardi torc vendran niquel reclus mat orden alej maner educ reclus clas medi convirt perr prisio

## 2.2 Preprocesamiento estilo XGBoost

Ahora hacemos el preprocesamiento que usa el modelo de XGBoost:
* limpieza general
* tokenización con spaCy
* lematización
* eliminación de stopwords

Luego dividimos el dataset en train y validation para poder evaluar el ensamble en el mismo conjunto que usamos para XGBoost original.

Esto nos da:
* X_proc — texto lematizado completo
* X_test_proc — texto lematizado del test
* X_train_text, X_val_text, y_train, y_val

In [ ]:


import re
import nltk
from nltk.corpus import stopwords
import spacy
from sklearn.model_selection import train_test_split

# Descargar recursos
nltk.download("stopwords")
stop_es = set(stopwords.words("spanish"))

# Cargar modelo de spaCy en español
!python -m spacy download es_core_news_sm > /dev/null
nlp = spacy.load("es_core_news_sm")

def clean_text(text):
    text = str(text).lower()
    text = re.sub(r"<[^>]+>", " ", text)            # eliminar HTML
    text = re.sub(r"[^a-záéíóúüñ\s]", " ", text)    # solo letras y espacios
    text = re.sub(r"\s+", " ", text)                # espacios múltiples
    return text.strip()

def tokenize_and_lemmatize(text):
    text = clean_text(text)
    doc = nlp(text)

    tokens = [
        token.lemma_.lower()
        for token in doc
        if token.is_alpha
        and token.lemma_.lower() not in stop_es
        and len(token.lemma_) > 2
    ]
    return tokens

def preprocess_to_string(text):
    return " ".join(tokenize_and_lemmatize(text))

# Texto crudo y etiquetas
X_raw = train_df["review_es"]
y     = train_df["sentimiento"]

print("Iniciando preprocesamiento estilo XGBoost (spaCy)...")
X_proc = X_raw.apply(preprocess_to_string)

print("Preprocesando test para XGBoost...")
X_test_raw  = test_df["review_es"]
X_test_proc = X_test_raw.apply(preprocess_to_string)

print("Separando datos de train en entrenamiento y validación...")
X_train_text, X_val_text, y_train, y_val = train_test_split(
    X_proc,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

print("Tamaño X_train_text:", len(X_train_text))
print("Tamaño X_val_text  :", len(X_val_text))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Iniciando preprocesamiento estilo XGBoost (spaCy)...
Preprocesando test para XGBoost...
Separando datos de train en entrenamiento y validación...
Tamaño X_train_text: 40000
Tamaño X_val_text  : 10000


## 3. Construir el conjunto de validación común

Necesitamos comparar NB, RF, XGB y el ensamble en una misma porción del dataset.
Usamos los índices de validación del split anterior, y construimos:
* X_val_nb → validación preprocesada con NB/RF
* X_val_xgb_text → validación preprocesada con spaCy
* X_val_xgb → validación vectorizada en TF–IDF con el vectorizador del XGB entrenado
* y_val_bin → etiquetas binarias (0=negativo, 1=positivo)

In [ ]:
# Índices de validación y etiquetas binarias

# Índices de las filas que quedaron en VALIDACIÓN en la notebook de XGBoost
idx_val = X_val_text.index

# Textos para cada modelo:
# - NB y RF: usan texto preprocesado con preprocess_text_stem
X_val_nb = X_nb_all.loc[idx_val]

# - XGBoost: usa el texto lematizado spaCy que ya teníamos en X_proc
X_val_xgb_text = X_proc.loc[idx_val]

# Etiquetas verdaderas (0 = negativo, 1 = positivo)
y_val_bin = y_val.map({"negativo": 0, "positivo": 1})

print("Tamaño validación NB/RF:", X_val_nb.shape)
print("Tamaño validación XGB  :", X_val_xgb_text.shape)
print("Tamaño y_val_bin        :", y_val_bin.shape)

# Matriz TF-IDF de validación para XGBoost
X_val_xgb = tfidf_xgb.transform(X_val_xgb_text)
X_val_xgb.shape


Tamaño validación NB/RF: (10000,)
Tamaño validación XGB  : (10000,)
Tamaño y_val_bin        : (10000,)


(10000, 78832)

## 4. Métricas de cada modelo individual (en el mismo validation)

En esta sección medimos el F1-score de cada uno de los tres modelos (NB, RF, XGB) sobre el mismo conjunto de validación.

Esto nos permite comparar después si el ensamble mejora algo respecto a los modelos individuales.

In [ ]:
# 4.5 Evaluación de cada modelo individual en el mismo set de validación

from sklearn.metrics import f1_score, classification_report

# --- Naive Bayes ---
y_val_pred_nb = nb_model.predict(X_val_nb)
f1_nb = f1_score(y_val_bin, y_val_pred_nb)
print(f"F1 (validación) Naive Bayes: {f1_nb:.4f}")

# --- Random Forest ---
y_val_pred_rf = rf_model.predict(X_val_nb)
f1_rf = f1_score(y_val_bin, y_val_pred_rf)
print(f"F1 (validación) Random Forest: {f1_rf:.4f}")

# --- XGBoost ---
y_val_pred_xgb = xgb_model.predict(X_val_xgb)
f1_xgb = f1_score(y_val_bin, y_val_pred_xgb)
print(f"F1 (validación) XGBoost: {f1_xgb:.4f}")


F1 (validación) Naive Bayes: 0.8816
F1 (validación) Random Forest: 0.8687
F1 (validación) XGBoost: 0.8731


In [ ]:
print("\nReporte Naive Bayes:\n",  classification_report(y_val_bin, y_val_pred_nb))
print("\nReporte Random Forest:\n", classification_report(y_val_bin, y_val_pred_rf))
print("\nReporte XGBoost:\n",      classification_report(y_val_bin, y_val_pred_xgb))


Reporte Naive Bayes:
               precision    recall  f1-score   support

           0       0.88      0.88      0.88      5000
           1       0.88      0.89      0.88      5000

    accuracy                           0.88     10000
   macro avg       0.88      0.88      0.88     10000
weighted avg       0.88      0.88      0.88     10000


Reporte Random Forest:
               precision    recall  f1-score   support

           0       0.91      0.80      0.85      5000
           1       0.82      0.92      0.87      5000

    accuracy                           0.86     10000
   macro avg       0.87      0.86      0.86     10000
weighted avg       0.87      0.86      0.86     10000


Reporte XGBoost:
               precision    recall  f1-score   support

           0       0.91      0.82      0.86      5000
           1       0.84      0.91      0.87      5000

    accuracy                           0.87     10000
   macro avg       0.87      0.87      0.87     10000
weighte

## 5.Ensamble por promedio de probabilidades (soft voting)

Cada modelo produce una probabilidad de que una review sea positiva.
El ensamble consiste en:
1. obtener la probabilidad de NB
2. obtener la probabilidad de RF
3. obtener la probabilidad de XGB
4. promediarlas
5. usar umbral 0.5 → positivo / negativo

Finalmente medimos el F1 del ensamble para ver si mejoró frente a los modelos individuales.

In [ ]:
# 4.6 Ensamble manual (soft voting)

import numpy as np

# Probabilidades de clase positiva (1) en validación
proba_nb  = nb_model.predict_proba(X_val_nb)[:, 1]
proba_rf  = rf_model.predict_proba(X_val_nb)[:, 1]
proba_xgb = xgb_model.predict_proba(X_val_xgb)[:, 1]

# Promedio simple de las tres probabilidades
proba_ens = (proba_nb + proba_rf + proba_xgb) / 3.0

# Umbral 0.5 → clase positiva
y_val_pred_ens = (proba_ens >= 0.5).astype(int)

f1_ens = f1_score(y_val_bin, y_val_pred_ens)
print(f"F1 (validación) ENSAMBLE (soft voting): {f1_ens:.4f}")

print("\nReporte de clasificación (Ensamble):\n")
print(classification_report(y_val_bin, y_val_pred_ens, target_names=["negativo", "positivo"]))


F1 (validación) ENSAMBLE (soft voting): 0.9059

Reporte de clasificación (Ensamble):

              precision    recall  f1-score   support

    negativo       0.92      0.88      0.90      5000
    positivo       0.88      0.93      0.91      5000

    accuracy                           0.90     10000
   macro avg       0.90      0.90      0.90     10000
weighted avg       0.90      0.90      0.90     10000



## 6. Submission del ensamble para Kaggle

Finalmente aplicamos el ensamble al dataset test.csv.
Repetimos el mismo esquema que en validación:
* NB y RF usan X_nb_test
* XGBoost usa tfidf_xgb.transform(X_test_proc)

Construimos un DataFrame con columnas ID y sentimiento y generamos el archivo:

In [ ]:
# Predicciones del ensamble sobre el test de Kaggle

# --- Entradas para cada modelo ---
X_test_nb   = X_nb_test                      # texto preprocesado NLTK
X_test_xgb  = tfidf_xgb.transform(X_test_proc)   # TF-IDF spaCy para XGB

# --- Probabilidades de cada modelo ---
proba_nb_test  = nb_model.predict_proba(X_test_nb)[:, 1]
proba_rf_test  = rf_model.predict_proba(X_test_nb)[:, 1]
proba_xgb_test = xgb_model.predict_proba(X_test_xgb)[:, 1]

# --- Promedio de probabilidades (soft voting) ---
proba_ens_test = (proba_nb_test + proba_rf_test + proba_xgb_test) / 3.0
y_test_pred_ens = (proba_ens_test >= 0.5).astype(int)

# Pasamos de 0/1 a "negativo"/"positivo"
pred_labels_ens = np.where(y_test_pred_ens == 1, "positivo", "negativo")

submission_ens = pd.DataFrame({
    "ID": test_df["ID"],
    "sentimiento": pred_labels_ens
})

submission_ens_filename = "submission_ensemble_soft.csv"
submission_ens.to_csv(submission_ens_filename, index=False)

print(f"Archivo de submission generado: {submission_ens_filename}")
submission_ens.head()


Archivo de submission generado: submission_ensemble_soft.csv


,ID,sentimiento
0,60000,negativo
1,60001,negativo
2,60002,negativo
3,60003,negativo
4,60004,negativo
